In [1]:
'This is my gensim with LDA'
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pandas as pd
import spacy
import numpy as np
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim



In [2]:
#import my data
df = pd.read_excel("Mentor_CSAT.xlsx")
df.head()

,Unnamed: 0,activityid,Lastdate,SentDate,StudentName,PMName,PMOperationsID,PathID,PipePathwayID,StudentType,...,"To help us better understand your experience, please explain the reasons for your satisfaction ratin",My Mentor shares useful information.,My Mentor responds to me within 24 hours.,My Mentor helps me get questions and issues resolved.,My Mentor cares about me and my success.,Sent Date,Semester,% Communicated,% Not Communicated,"Overall, how satisfied are you with BYU-Pathway Mentoring?"
0,0,F22B8E8C-476D-EE11-8DF0-00224808045A,2023-10-17 23:47:33,10/16/2023,Hnin Wai Oo,Maria Carmela Villenas Gaela,10260,22822379,22822379,Matriculation,...,She's really kind to me,Mostly true,Moderately true,Slightly true,Moderately true,2023-10-16,Fall 2023,Communicated,NaN,Satisfied
1,1,C049BF3C-0F10-EE11-8F6D-0022480A40EB,2023-06-21 08:40:19,6/20/2023,Adrian James Oriewo,Susan Kapiya Chitembo,10121,22560197,22560197,Online Degree - BYU-Idaho,...,The Peer Mentor always encourages me to keep o...,Definitely true,Definitely true,Definitely true,Definitely true,2023-06-20,Spring 2023,Communicated,NaN,Very satisfied
2,2,15D312C2-80F9-ED11-8849-0022480AE0F6,2023-05-23 15:44:56,5/22/2023,Marie Angelene Empleo Diaz,Julie Anne Gollayan Alberto Espino,10503,22525995,22525995,PathwayConnect,...,She replies whenever I need her help.,Moderately true,Moderately true,Moderately true,Moderately true,2023-05-22,Spring 2023,Communicated,NaN,Very satisfied
3,3,8F082B81-0C25-EE11-9CBE-0022480AE0F6,2023-07-18 01:43:35,7/17/2023,Gabart Eugenio Jean Louis,Michael Aboagye Adu,10389,20062864,20062864,PathwayConnect,...,Best mentor ever thus far,Definitely true,Definitely true,Definitely true,Definitely true,2023-07-17,Spring 2023,Communicated,NaN,Very satisfied
4,4,264787A9-EC98-EE11-BE37-0022480AE1C8,2023-12-12 12:47:48,12/11/2023,Nomathamsanqa Maho,Evans Nana Baah Ankomah,10243,13242085,13242085,Online Degree - Ensign College,...,NaN,NaN,NaN,NaN,NaN,2023-12-11,Fall 2023,NaN,Not Communicated,NaN


In [3]:
stopwords = stopwords.words('english')

In [4]:
data = df['To help us better understand your experience, please explain the reasons for your satisfaction ratin'].values



In [5]:
def lemmatization(texts, allowed_posttags = ["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(str(text))
        new_text = []
        for token in doc:
            if token.pos_ in allowed_posttags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        if final != '':
            texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print(lemmatized_texts[0:5])

['really kind', 'always encourage keep push foward study most importantly share scripture strengthen', 'reply need help', 'good mentor ever thus far', 'very satisfied peer mentor always available need always answer question very important']


In [6]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text,deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)
print(data_words[0:5])

[['really', 'kind'], ['always', 'encourage', 'keep', 'push', 'foward', 'study', 'most', 'importantly', 'share', 'scripture', 'strengthen'], ['reply', 'need', 'help'], ['good', 'mentor', 'ever', 'thus', 'far'], ['very', 'satisfied', 'peer', 'mentor', 'always', 'available', 'need', 'always', 'answer', 'question', 'very', 'important']]


In [7]:
#Bigrams and Trigrams
bigrams_phrases = gensim.models.Phrases(data_words,min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigrams_phrases[data_words], threshold=100)
bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return ([bigram[doc] for doc in texts])

def make_trigram(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigram(data_bigrams)


print(data_bigrams_trigrams[0:5])




[['really', 'kind'], ['always', 'encourage', 'keep', 'push', 'foward', 'study', 'most_importantly', 'share', 'scripture', 'strengthen'], ['reply', 'need', 'help'], ['good', 'mentor', 'ever', 'thus_far'], ['very', 'satisfied', 'peer', 'mentor', 'always', 'available', 'need', 'always', 'answer', 'question', 'very', 'important']]


In [8]:
#TF-IDF Removal
from gensim.models import TfidfModel


id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[0])

tfidf = TfidfModel(corpus=corpus, id2word=id2word)
low_value = 0.03
words = []
words_missing_in_tfidif = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidif
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidif = [id for id in bow_ids if id not in tfidf_ids]

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidif]
    corpus[i] = new_bow



[(0, 1), (1, 1)]


In [9]:
# id2word = corpora.Dictionary(data_words)

# corpus = []
# for text in data_words:
#     new = id2word.doc2bow(text)
#     corpus.append(new)

# print(corpus[0:5])
# word = id2word[[0][0]]
# print(word)

In [10]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus[:-1], id2word=id2word, num_topics= 10, random_state=100,update_every=1,chunksize=100,passes=10, alpha="auto")

In [11]:
#save model
'''lda_model.save("models/gensim_lda_model")
'''

'lda_model.save("models/gensim_lda_model")\n'

In [12]:
#load model
'''new_model = gensim.models.ldamodel.LdaModel.load("models/gensim_lda_model)'''
'''run everything with the new model'''

'run everything with the new model'

In [13]:
test_doc = corpus[-1]
vector = lda_model[test_doc]
print(vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)

new_vector = Sort(vector)
print(new_vector)

[(0, 0.25830525), (1, 0.0924566), (2, 0.060386293), (3, 0.062306717), (4, 0.08351366), (5, 0.100091554), (6, 0.089624554), (7, 0.081632815), (8, 0.08582368), (9, 0.0858589)]
[(0, 0.25830525), (5, 0.100091554), (1, 0.0924566), (6, 0.089624554), (9, 0.0858589), (8, 0.08582368), (4, 0.08351366), (7, 0.081632815), (3, 0.062306717), (2, 0.060386293)]


In [14]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

c:\Python311\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Python311\Lib\site-packages\sklearn\manifold\_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.390610  0.264473       1        1  24.357458
8      0.117136  0.432489       2        1  11.269693
6     -0.348635 -0.258519       3        1  11.125925
4     -0.248517  0.421896       4        1  10.574363
1      0.200879 -0.432469       5        1   8.035680
3     -0.171118  0.150804       6        1   7.942688
2     -0.111732 -0.417956       7        1   7.363073
9     -0.475309  0.026200       8        1   7.063292
7      0.180433 -0.082416       9        1   6.625254
5      0.466253 -0.104502      10        1   5.642574, topic_info=             Term          Freq         Total Category  logprob  loglift
17         mentor  10008.000000  10008.000000  Default  30.0000  30.0000
12           help  14506.000000  14506.000000  Default  29.0000  29.0000
2          always  12997.000000  12997.000000  Default  28.0000  28.0000
25           very   6195.000000   6195.000000  Default  27.0000  27.0000
23       question   5340.000000   5340.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
660      strength     76.784929     77.667672  Topic10  -5.4931   2.8634
1088      welfare     74.001424     74.884288  Topic10  -5.5300   2.8630
260   performance     73.516115     74.398893  Topic10  -5.5366   2.8629
972        lovely     70.706878     71.589621  Topic10  -5.5756   2.8624
831          such     63.450569     64.333272  Topic10  -5.6838   2.8610

[330 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
187       8  0.998675      able
641       8  0.998125  academic
58        5  0.988094   account
253       2  0.997247   achieve
562       6  0.993850  actually
...     ...       ...       ...
167       2  0.999033      work
551       8  0.994751     worry
335       3  0.999196     write
1180      6  0.995226      year
612       5  0.994253       yet

[300 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 9, 7, 5, 2, 4, 3, 10, 8, 6])